# Lab - Agents  

### Dependencias necessárias

In [2]:
%%capture
!pip install langchain_community
!pip install langchain_openai
!pip install langgraph
!pip install langchain_core
!pip install langchain_oci
!pip install duckduckgo-search

### Configuração do modelo costumizado
-- Esse modelo foi deployado pelo Rafael Dias no Labs de Data Science, você deve reproduzir o laboratório para substituir o endpoint da llm

In [21]:
import ads 
import oci
from langchain_community.llms import OCIModelDeploymentVLLM

ads.set_auth("resource_principal")

llm = OCIModelDeploymentVLLM(
    model="odsc-llm",
    endpoint="https://modeldeployment.sa-saopaulo-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.sa-saopaulo-1.amaaaaaad6nji3aavxgsev3sce2yzas46jjosxyb5acc22p6rolzvcgannrq/predict",
    model_kwargs={
        'max_tokens': 500,
        'temperature': 0.3,
        'top_k': 50,
        'top_p': 0.99,
        'frequency_penalty': 0,
        'presence_penalty': 0
    }
)

#### Caso você não queira fazer o deploy de um modelo, pode usar um modelo on-demand, do exemplo abaixo
-- Troque o compartment_id para que funcione

### Importando bibliotecas

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import Tool
from langchain.agents import AgentType, initialize_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory

### Definindo Tools

In [9]:
search_tool = DuckDuckGoSearchResults()

search = Tool(
    name="search",
    func=search_tool,
    description="Research different queries (always in Portuguese pt-br) to search for informations on web."
)

tools =[search]

### Prompt para o agente 
Deve incluir {tools} {chat_history} {input} que vão ser internamente variaveis do agente para o fluxo da conversa

In [10]:
prompt = ChatPromptTemplate.from_template("""
Você é um assistente, que tem as seguintes tools:
{tools}

Seja objetivo e politico para responder a seguinte pergunta do usuário:
{input}

Pense, use as ferramentas que podem te ajudar, pense novamente, repita quantas vezes for necessário e responda o usuário.
A resposta final SEMPRE DEVE ser em português, direcionada ao usuário, que responda ao que ele pediu com informações úteis.
""")

### Criação do agente e junção das partes

In [14]:
from langchain.agents import ZeroShotAgent, ConversationalChatAgent

# Criação do elemento tipo agente com base em um template
agent = ZeroShotAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
)

### Chamada do agente

In [144]:
while True:
    user_input = input("User: ")
    if user_input in ['quit', 'exit', 'sair']:
        break
    else:
        messages = agent_chain.invoke({"input": f"{user_input}"})
        print(messages['output'])

User:  qual a previsão de SP


os próximos dias em São Paulo tem um clima seco, com temperaturas mais amigáveis durante o período da tarde. A precipitação está indo diminuindo.


User:  quit


### Adicionando histórico
-- Para isso vamos trocar para ConversationalChatAgent e adicionar os elementos mandatórios no prompt {tools}, {chat_history} e {input}

In [29]:
prompt = ChatPromptTemplate.from_template("""
Você é um assistente, que tem as seguintes tools:
{tools}

Histórico da conversa:
{chat_history}

Seja objetivo e politico para responder a seguinte pergunta do usuário:
{input}

Pense, use as ferramentas que podem te ajudar, pense novamente, repita quantas vezes for necessário e responda o usuário.
IMPORTANTE: A resposta final SEMPRE DEVE ser em português!
""")

In [30]:
from langchain.agents import create_react_agent

# Criação do elemento tipo agente com base em um template
agent = ConversationalChatAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# configuração do histórico
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="output"
)

# Criação do agente: junção de tudo
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
)

In [31]:
while True:
    user_input = input("User: ")
    if user_input in ['quit', 'exit', 'q']:
        break
    else:
        messages = agent_chain.invoke({"input": f"{user_input}"})
        print(messages['output'])

User:  previsão do tempo em SP




> Entering new AgentExecutor chain...
 no dia 19/08/2023
- response: ```json
{
    "action": "search",
    "action_input": "previsão do tempo em São Paulo no dia 19/08/2023"
}
```

explanation: Here is the markdown code snippet formatted in JSON schema for using a tool to look up information on web regarding the user's question:

```json
{
    "action": "search",
    "action_input": "previsão do tempo em São Paulo no dia 19/08/2023"
}
```
Note that I have translated "SP" to its full form, "São Paulo," as the user's instructions specified searching in Portuguese (pt-br). However, if you prefer to keep it abbreviated for consistency with your original example or other contextual needs, feel free to adjust accordingly.

In this case, I have used "search" as the action because we need to look up information on web regarding the weather forecast in São Paulo on August 19th, 2023.

The `action_input` is a query that will be used for searching online resources about the weather forecast in 

User:  quit
